In [1]:
# python3 -m venv .venv
# source .venv/bin/activate
# python -m pip install --upgrade pip
# pip install -r requirements.txt

# requirements.txt
## ipykernel
## ortools
## pandas
## numpy
## folium
## geopandas
## matplotlib
## networkx
## jupyter
## plotly
## pytest
## hypothesis
## fastapi
## uvicorn
## sqlalchemy
## streamlit
## openrouteservice
## tqdm
## osmnx
## networkx
## scikit-learn

In [2]:
import osmnx as ox
import networkx as nx

G = ox.graph_from_place('Mannheim, Germany', network_type='drive')

In [3]:
def get_truck_path_and_length(lat1, lon1, lat2, lon2):
    orig_node = ox.distance.nearest_nodes(G, lon1, lat1)
    dest_node = ox.distance.nearest_nodes(G, lon2, lat2)
    try:
        path = nx.shortest_path(G, orig_node, dest_node, weight='length')
        length = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
        return length, path
    except nx.NetworkXNoPath:
        return float('inf'), []

In [4]:
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("altglascontainer-mannheim-clean.csv", delimiter=',')
locations = list(zip(df["Latitude"], df["Longitude"]))

distance_matrix = []
path_matrix = []

for i, (lat1, lon1) in enumerate(tqdm(locations, desc="Outer loop")):
    break
    dist_row = []
    path_row = []
    for j, (lat2, lon2) in enumerate(tqdm(locations, desc="Inner loop", leave=False)):
        dist, path = get_truck_path_and_length(lat1, lon1, lat2, lon2)
        if dist == float('inf'):
            dist_row.append(-1)
            path_row.append([])
        else:
            dist_row.append(int(round(dist)))
            path_row.append(path)
    distance_matrix.append(dist_row)
    path_matrix.append(path_row)



distance_matrix_df = pd.DataFrame(distance_matrix)
path_matrix_df = pd.DataFrame(path_matrix)



Outer loop:   0%|          | 0/293 [00:00<?, ?it/s]


In [5]:
#distance_matrix_df.to_csv("distance_matrix.csv", index=False)
#path_matrix_df.to_csv("path_matrix.csv", index=False)

In [6]:
def nodes_to_coords(G, path):
    return [(G.nodes[n]['y'], G.nodes[n]['x']) for n in path]


In [7]:
import folium
import ast



def visualize_path_between_containers(start_index, end_index):
    start_lat, start_lon = locations[start_index]
    end_lat, end_lon = locations[end_index]

    example_path_nodes = path_matrix.iloc[start_index, end_index]
    if not example_path_nodes:
        print(f"No path found between container {start_index} and {end_index}.")
        return None

    example_path_coords = nodes_to_coords(G, example_path_nodes)
    m = folium.Map(location=[start_lat, start_lon], zoom_start=14)

    folium.Marker(location=[start_lat, start_lon], popup=f"Start: {start_index}", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=[end_lat, end_lon], popup=f"End: {end_index}", icon=folium.Icon(color='red')).add_to(m)

    folium.PolyLine(example_path_coords, color='blue', weight=5, opacity=0.7).add_to(m)

    return m


In [9]:
distance_matrix_df = pd.read_csv("distance_matrix.csv")
path_matrix_df = pd.read_csv("path_matrix.csv")

path_matrix = path_matrix_df.copy()
for col in path_matrix.columns:
    path_matrix[col] = path_matrix[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and x != '[]' else [])

distance_matrix = distance_matrix_df.values.tolist()
m = visualize_path_between_containers(261, 25)
m